In [1]:
!pip install keras-tuner

: 

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
from tensorflow.keras.datasets import fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [22]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
      filters = hp.Int('conv_1_filter' , min_value =64, max_value = 128, step = 16),
      kernel_size = hp.Choice('conv_1_kernel' , values = [3,5]),
      activation = 'relu',
      input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
      filters = hp.Int('conv_2_filter' , min_value =32, max_value = 64, step = 16),
      kernel_size = hp.Choice('conv_2_kernel' , values = [3,5]),
      activation = 'relu',

      ),
      keras.layers.Conv2D(
      filters = hp.Int('conv_3_filter' , min_value =32, max_value = 128, step = 16),
      kernel_size = hp.Choice('conv_3_kernel' , values = [3,5]),
      activation = 'relu',

      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int("dense_1_units",min_value = 32, max_value = 128, step=16),
          activation = 'relu'
      ),
      keras.layers.Dense(units = 10, activation = 'softmax')
  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice("learning_rate" , values = [1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy' ])
  return model


In [23]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [24]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,
                            directory = 'output',
                            project_name = "Fashion MNIST")
tuner_search.search(train_images,train_labels,epochs = 3,validation_split = 0.1)

Trial 5 Complete [00h 00m 36s]
val_accuracy: 0.8326666951179504

Best val_accuracy So Far: 0.9068333506584167
Total elapsed time: 00h 02m 58s


In [26]:
tuner_search.get_best_hyperparameters()[0].values

{'conv_1_filter': 64,
 'conv_1_kernel': 5,
 'conv_2_filter': 32,
 'conv_2_kernel': 3,
 'conv_3_filter': 32,
 'conv_3_kernel': 3,
 'dense_1_units': 80,
 'learning_rate': 0.001}

In [27]:
model = tuner_search.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 64)          │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 20, 20, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 80)                  │       1,024,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             810 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,054,266 (4.02 MB)

 Trainable params: 1,054,266 (4.02 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.fit(train_images, train_labels, validation_split = 0.1, epochs= 3)

Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9345 - loss: 0.1798 - val_accuracy: 0.9010 - val_loss: 0.2641
Epoch 2/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9461 - loss: 0.1443 - val_accuracy: 0.9092 - val_loss: 0.2842
Epoch 3/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9599 - loss: 0.1095 - val_accuracy: 0.9142 - val_loss: 0.2740


In [31]:
labels_pred = model.predict(test_images)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [32]:
labels_pred

array([[7.73353874e-13, 6.87606574e-14, 1.00924022e-12, ...,
        9.95176379e-07, 1.41966639e-13, 9.99999046e-01],
       [1.32598925e-05, 6.72808231e-14, 9.99837518e-01, ...,
        9.94262181e-15, 5.39506482e-13, 4.47101772e-10],
       [9.98653764e-17, 1.00000000e+00, 1.67488347e-22, ...,
        1.51216007e-24, 9.85814723e-25, 2.15251002e-25],
       ...,
       [1.89318131e-10, 2.70067104e-14, 6.76467840e-11, ...,
        9.08779752e-14, 1.00000000e+00, 9.16221034e-16],
       [3.83052287e-17, 1.00000000e+00, 4.58414384e-20, ...,
        1.26241221e-23, 1.05781311e-19, 6.65791988e-19],
       [2.17541061e-08, 2.30175989e-10, 1.40177541e-08, ...,
        5.48860726e-06, 5.95651454e-06, 1.05762183e-07]], dtype=float32)

In [36]:

predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predicted_classes)
print(f'Manual Accuracy Score: {accuracy * 100:.2f}%')



313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Manual Accuracy Score: 91.00%
